In [1]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import calendar
import undetected_chromedriver as uc
import locale

In [ ]:
# json_pages_info = {"pages": []}

In [3]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}

pages = 10

re = False

for page in range(0, pages):

	first = True
	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.theclinic.cl&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'
				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
			articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

			for article in articles:
				print("revisando artículo...")
				try: 
					originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
					for es, en in months.items():
						if es in originalDate:
								originalDate = originalDate.replace(es, en)
								break
						
					date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

					if (date_epoch < 1573786800 or date_epoch > 1702782000):
						print("La noticia no corresponde a la fecha solicitada")
						continue

					# check if the article is from El Mostrador
					if (not ("www.theclinic.cl" in article.text)):
						print("El artículo no es de The Clinic")
						continue

					try:
						title = article.find_element(By.TAG_NAME, "h3").text
						description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
						link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

						link_info = {
							"newscast" : "The Clinic",
							"title": title,
							"description": description,
							"category": "The site does not provide a category",
							"date": originalDate,
							"image_link": "not found initially",
							"author": "not found initially",
							"link": link,
						}

						print("información incluida!")
						json_pages_info["pages"].append(link_info)

					except Exception as e:
						print(f"Error al extraer información del artículo: {e}")
						continue

				except Exception as e:
					print(f"Error al procesar el artículo: {e}")
					continue
		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/the_clinic.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


# with open(f"output.html", "w", encoding="utf-8") as f:
# 	f.write(driver.page_source)

revisando artículo...
información incluida!
revisando artículo...
Error al procesar el artículo: Message: no such element: Unable to locate element: {"method":"css selector","selector":".YrbPuc"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x1051af3+62339]
	GetHandleVerifier [0x0x1051b34+62404]
	(No symbol) [0x0xe92123]
	(No symbol) [0x0xeda86e]
	(No symbol) [0x0xedac0b]
	(No symbol) [0x0xed0081]
	(No symbol) [0x0xeff404]
	(No symbol) [0x0xecffa4]
	(No symbol) [0x0xeff634]
	(No symbol) [0x0xf207a3]
	(No symbol) [0x0xeff1b6]
	(No symbol) [0x0xece7a2]
	(No symbol) [0x0xecf644]
	GetHandleVerifier [0x0x12c6683+2637587]
	GetHandleVerifier [0x0x12c1a8a+2618138]
	GetHandleVerifier [0x0x107856a+220666]
	GetHandleVerifier [0x0x1068998+156200]
	GetHandleVerifier [0x0x106f12d+182717]
	GetHandleVerifier [0x0x1059a38+94920]
	

# Content Extraction